# Modelo de regresión con redes neuronales

In [15]:
!pip install scikeras
!pip install --upgrade category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Input, Reshape
from keras import backend as K
from keras import applications
from keras.utils.vis_utils import plot_model

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.pipeline import Pipeline

from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
import category_encoders as ce

print('Tensorflow version:', tf.__version__)

Tensorflow version: 2.9.2


In [17]:
def get_metrics(Y_test, predictions):
  print('MAE: ', mean_absolute_error(Y_test, predictions))
  print('RMSE: ', np.sqrt(mean_squared_error(Y_test, predictions)))
  print('r2 : ', r2_score(Y_test, predictions))
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()

Para el modelo obtenido con redes neuronales se usarán los datasets creados en la etapa de preprocesamiento. Estos cuentan con transforaciones OneHot y binary, así como incluyendo las columnas de Manufacturer y Model y aplicando normalización estándar y min-max. Se aplicará variación de hiperparámetros con el fin de conseguir el mejor modelo para cada dataset y  al final se compararán por medio del RMSE en el conjunto de validación. \
Para los modelos obtenidos con redes neuronales se obtendrán **4 modelos**, uno por cada dataset y por cada dataset se hará variación de hiperparámetros. \
Con base a esto se selecciona el mejor modelo de los 4 y se entrará a evaluar en el conjunto de test.

## Dataset sin columnas de Manufacturer y Model

In [18]:
# Dataset one hot encoding con Standard sclaer
X_train_one_hot_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/standard/train/X_train_one_hot_encoding_standard.csv')
y_train_one_hot_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/standard/train/y_train_one_hot_encoding_standard.csv')

X_val_one_hot_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/standard/val/X_val_one_hot_encoding_standard.csv')
y_val_one_hot_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/standard/val/y_val_one_hot_encoding_standard.csv')

X_test_one_hot_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/standard/test/X_test_one_hot_encoding_standard.csv')
y_test_one_hot_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/standard/test/y_test_one_hot_encoding_standard.csv')

In [26]:
# Dataset one hot encoding con Min Max Scaler
X_train_one_hot_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/min_max/train/X_train_one_hot_encoding_min_max.csv')
y_train_one_hot_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/min_max/train/y_train_one_hot_encoding_min_max.csv')

X_val_one_hot_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/min_max/val/X_val_one_hot_encoding_min_max.csv')
y_val_one_hot_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/min_max/val/y_val_one_hot_encoding_min_max.csv')

X_test_one_hot_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/min_max/test/X_test_one_hot_encoding_min_max.csv')
y_test_one_hot_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/one_hot_encoding/min_max/test/y_test_one_hot_encoding_min_max.csv')

A continuación se entrenará un primer modelo base con el dataset que tiene Standard Scaler.

### Modelo base

In [22]:
def get_model(meta, activation, n_layers, hidden_layer_1, loss_f, hidden_layer_2 = None, hidden_layer_3 = None):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    dims = [hidden_layer_1, hidden_layer_2, hidden_layer_3]

    model = Sequential(name="Red_CV")
    model.add(Dense(n_features_in_, input_shape=X_shape_[1:]))

    for i in range(n_layers):
        model.add(Dense(dims[i], activation=activation))
    
    model.add(Dense(1, activation='linear'))
    model.compile(loss=loss_f, optimizer="adam") 
    return model
clf = KerasRegressor(
    get_model,
    loss="mse",
    loss_f='mse',
    n_layers=3,
    hidden_layer_1=512, 
    hidden_layer_2=512,
    hidden_layer_3=512,
    metrics=['mae', 'mse'],
    epochs=50,
    activation="relu"
)

# clf
clf.fit(X_train_one_hot_encoding_standard, y_train_one_hot_encoding_standard,
        validation_data=(    
                X_val_one_hot_encoding_standard, 
                y_val_one_hot_encoding_standard
            ))

predictions_train_base_model = clf.predict(X_train_one_hot_encoding_standard)
predictions_val_base_model = clf.predict(X_val_one_hot_encoding_standard)

get_metrics(y_train_one_hot_encoding_standard, predictions_train_base_model)
get_metrics(y_val_one_hot_encoding_standard, predictions_val_base_model)

Epoch 1/50
229/229 [==============================] - 1s 4ms/step - loss: 236751792.0000 - val_loss: 378916044800.0000
Epoch 2/50
229/229 [==============================] - 1s 4ms/step - loss: 123506552.0000 - val_loss: 378887831552.0000
Epoch 3/50
229/229 [==============================] - 1s 4ms/step - loss: 115416120.0000 - val_loss: 378903691264.0000
Epoch 4/50
229/229 [==============================] - 1s 4ms/step - loss: 111412312.0000 - val_loss: 378866171904.0000
Epoch 5/50
229/229 [==============================] - 1s 6ms/step - loss: 111367464.0000 - val_loss: 378916208640.0000
Epoch 6/50
229/229 [==============================] - 1s 4ms/step - loss: 109492400.0000 - val_loss: 378904412160.0000
Epoch 7/50
229/229 [==============================] - 1s 4ms/step - loss: 107532352.0000 - val_loss: 378893664256.0000
Epoch 8/50
229/229 [==============================] - 1s 4ms/step - loss: 107539912.0000 - val_loss: 378850148352.0000
Epoch 9/50
229/229 [============================

In [23]:
#Conjunto de entrenamiento
predictions_train_base_model = clf.predict(X_train_one_hot_encoding_standard)
#Conjunto de valdiación
predictions_val_base_model = clf.predict(X_val_one_hot_encoding_standard)

get_metrics(y_train_one_hot_encoding_standard, predictions_train_base_model)
get_metrics(y_val_one_hot_encoding_standard, predictions_val_base_model)

58/58 [==============================] - 0s 2ms/step
MAE:  4771.463253468846
RMSE:  7644.592216032139
r2 :  0.7732296504140259
MAE:  19669.523372403633
RMSE:  615544.5221150985
r2 :  -0.0009852308351550043


Con base a las métricas anteriores se observa evidencia de overfitting, ya que se obtuvo en desempeño inferior en el conjunto de validación comparado con el de entrenamiento.

Una vez encontrado el modelo base procederemos a realizar la variación de hiperparámetros. Para ello se hará uso de la librería RandomizedSearchCV.

### Variación de hiperparámetros

### Dataset con Standard Scaler

In [14]:
def get_model_mlp_classifier(meta, activation, n_layers, hidden_layers_list):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    dims = hidden_layers_list

    model = Sequential(name='Clasificador')

    #Capa de Entrada
    model.add(Input(shape=(X_shape_[1:]), name='Input_Layer'))


    #model.add(Dense(n_features_in_, input_shape=X_shape_[1:]))

    for i in range(n_layers):
        model.add(Dense(dims[i], activation=activation))
  
    model.add(Dense(1, activation='linear'))
    return model
#Búsqueda de hiperparámetros para el nuevo perceptrón multicapa    
final_params = {
    'hidden_layers_list':  [[100, 100], [200]*2, [200]*3, [256]*3, [200]*4],
    "epochs": [50, 100],
    'optimizer': ["adam", "sgd", "RMSprop"],
    "activation": ["relu", "sigmoid", "softmax"],
    'n_layers':[2, 2, 3, 3, 4],
    #'batch_size':[10, 50, 100, 200]
}
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='/content/logs'),
]
clf = KerasRegressor(
    get_model_mlp_classifier,
    loss="mse",
    n_layers=4,
    metrics=["mse", "mae"],
    epochs=100,
    activation="sigmoid",
    hidden_layers_list = [256]*4
)

grid = RandomizedSearchCV(clf, final_params, cv=3, n_iter=10, random_state=33)
grid.fit(X_train_one_hot_encoding_standard,
        y_train_one_hot_encoding_standard,
        callbacks=my_callbacks,
         validation_data=(    
                X_val_one_hot_encoding_standard, 
                y_val_one_hot_encoding_standard
            )
)

Epoch 1/100
153/153 [==============================] - 1s 5ms/step - loss: 593035328.0000 - mean_squared_error: 593035328.0000 - mean_absolute_error: 18323.2148 - val_loss: 380894380032.0000 - val_mean_squared_error: 380894380032.0000 - val_mean_absolute_error: 48705.4805
Epoch 2/100
153/153 [==============================] - 1s 4ms/step - loss: 457086272.0000 - mean_squared_error: 457086272.0000 - mean_absolute_error: 16444.5449 - val_loss: 379121565696.0000 - val_mean_squared_error: 379121565696.0000 - val_mean_absolute_error: 26229.6992
Epoch 3/100
153/153 [==============================] - 1s 4ms/step - loss: 336471328.0000 - mean_squared_error: 336471328.0000 - mean_absolute_error: 13529.8965 - val_loss: 379399536640.0000 - val_mean_squared_error: 379399536640.0000 - val_mean_absolute_error: 29985.2422
Epoch 4/100
153/153 [==============================] - 1s 4ms/step - loss: 379869120.0000 - mean_squared_error: 379869120.0000 - mean_absolute_error: 14388.9756 - val_loss: 37924454

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

153/153 [==============================] - 1s 5ms/step - loss: 24230955008.0000 - mean_squared_error: 24230955008.0000 - mean_absolute_error: 97547.7891 - val_loss: 649114812416.0000 - val_mean_squared_error: 649114812416.0000 - val_mean_absolute_error: 520185.1250
Epoch 2/100
153/153 [==============================] - 1s 4ms/step - loss: 3038922900766720.0000 - mean_squared_error: 3038922900766720.0000 - mean_absolute_error: 31824168.0000 - val_loss: 40204372814069760.0000 - val_mean_squared_error: 40204372814069760.0000 - val_mean_absolute_error: 200509328.0000
Epoch 3/100
153/153 [==============================] - 1s 4ms/step - loss: 495519702447390982144.0000 - mean_squared_error: 495519702447390982144.0000 - mean_absolute_error: 12851783680.0000 - val_loss: 6555489720146348474368.0000 - val_mean_squared_error: 6555489720146348474368.0000 - val_mean_absolute_error: 80966000640.0000
Epoch 4/100
153/153 [==============================] - 1s 4ms/step - loss: 80771310206142227461177344

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

153/153 [==============================] - 1s 5ms/step - loss: 613057728.0000 - mean_squared_error: 613057728.0000 - mean_absolute_error: 19056.7441 - val_loss: 379618099200.0000 - val_mean_squared_error: 379618099200.0000 - val_mean_absolute_error: 33105.3281
Epoch 2/50
153/153 [==============================] - 1s 4ms/step - loss: 612593088.0000 - mean_squared_error: 612593088.0000 - mean_absolute_error: 19044.5371 - val_loss: 379617280000.0000 - val_mean_squared_error: 379617280000.0000 - val_mean_absolute_error: 33093.4453
Epoch 3/50
153/153 [==============================] - 1s 4ms/step - loss: 612122624.0000 - mean_squared_error: 612122624.0000 - mean_absolute_error: 19032.2773 - val_loss: 379616362496.0000 - val_mean_squared_error: 379616362496.0000 - val_mean_absolute_error: 33081.0430
Epoch 4/50
153/153 [==============================] - 1s 4ms/step - loss: 611625728.0000 - mean_squared_error: 611625728.0000 - mean_absolute_error: 19019.4102 - val_loss: 379615510528.0000 - val

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

153/153 [==============================] - 1s 5ms/step - loss: 543676928.0000 - mean_squared_error: 543676928.0000 - mean_absolute_error: 17776.9844 - val_loss: 379767980032.0000 - val_mean_squared_error: 379767980032.0000 - val_mean_absolute_error: 35341.6953
Epoch 2/100
153/153 [==============================] - 1s 4ms/step - loss: 340857600.0000 - mean_squared_error: 340857600.0000 - mean_absolute_error: 13579.6562 - val_loss: 379996045312.0000 - val_mean_squared_error: 379996045312.0000 - val_mean_absolute_error: 38391.8945
Epoch 3/100
153/153 [==============================] - 1s 4ms/step - loss: 829033792.0000 - mean_squared_error: 829033792.0000 - mean_absolute_error: 22794.5000 - val_loss: 378543472640.0000 - val_mean_squared_error: 378543472640.0000 - val_mean_absolute_error: 36784.6211
Epoch 4/100
153/153 [==============================] - 1s 4ms/step - loss: 746571328.0000 - mean_squared_error: 746571328.0000 - mean_absolute_error: 21300.0684 - val_loss: 381382361088.0000 - 

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
6 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 767, in fit
    **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 918, in _fit
    X, y = self._initialize(X, y)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 855, in _initialize
  

229/229 [==============================] - 2s 6ms/step - loss: 280480704.0000 - mean_squared_error: 280480704.0000 - mean_absolute_error: 11176.5166 - val_loss: 378699546624.0000 - val_mean_squared_error: 378699546624.0000 - val_mean_absolute_error: 25396.4844
Epoch 2/50
229/229 [==============================] - 1s 4ms/step - loss: 257909392.0000 - mean_squared_error: 257909392.0000 - mean_absolute_error: 10997.7100 - val_loss: 378717274112.0000 - val_mean_squared_error: 378717274112.0000 - val_mean_absolute_error: 25180.1777
Epoch 3/50
229/229 [==============================] - 1s 4ms/step - loss: 257873296.0000 - mean_squared_error: 257873296.0000 - mean_absolute_error: 11003.4473 - val_loss: 378745389056.0000 - val_mean_squared_error: 378745389056.0000 - val_mean_absolute_error: 24910.1016
Epoch 4/50
229/229 [==============================] - 1s 4ms/step - loss: 258030704.0000 - mean_squared_error: 258030704.0000 - mean_absolute_error: 10951.0869 - val_loss: 378715308032.0000 - val

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(activation='sigmoid', epochs=100, hidden_layers_list=[256, 256, 256, 256], loss='mse', metrics=['mse', 'mae'], model=<function get_model_mlp_classifier at 0x7f611240f4d0>, n_layers=4),
                   param_distributions={'activation': ['relu', 'sigmoid',
                                                       'softmax'],
                                        'epochs': [50, 100],
                                        'hidden_layers_list': [[100, 100],
                                                               [200, 200],
                                                               [200, 200, 200],
                                                               [256, 256, 256],
                                                               [200, 200, 200,
                                                                200]],
                                        'n_layers': [2, 2, 3, 3, 4],
               

In [15]:
best_model_standar_scaler = grid.best_estimator_

grid.best_params_

{'optimizer': 'sgd',
 'n_layers': 3,
 'hidden_layers_list': [200, 200, 200, 200],
 'epochs': 50,
 'activation': 'softmax'}

In [16]:
predictions_train_standard_scaler_keras_tuner = best_model_standar_scaler.predict(X_train_one_hot_encoding_standard)
predictions_val_standard_scaler_keras_tuner = best_model_standar_scaler.predict(X_val_one_hot_encoding_standard)

get_metrics(y_train_one_hot_encoding_standard,predictions_train_standard_scaler_keras_tuner)
get_metrics(y_val_one_hot_encoding_standard,predictions_val_standard_scaler_keras_tuner)

58/58 [==============================] - 0s 2ms/step
MAE:  11287.57530173963
RMSE:  16083.952765155593
r2 :  -0.003837001629903636
MAE:  25495.44702603026
RMSE:  615379.8349496782
r2 :  -0.000449681051579498


Con base a las métricas anteriores se observa evidencia de overfitting, ya que se obtuvo en desempeño inferior en el conjunto de validación comparado con el de entrenamiento.

### Dataset con Min-Max Scaler

In [17]:
grid.fit(X_train_one_hot_encoding_min_max,
        y_train_one_hot_encoding_min_max,
        callbacks=my_callbacks,
         validation_data=(    
                X_val_one_hot_encoding_min_max, 
                y_val_one_hot_encoding_min_max
            )
)

Epoch 1/100
153/153 [==============================] - 1s 5ms/step - loss: inf - mean_squared_error: inf - mean_absolute_error: 13752294516524580864.0000 - val_loss: nan - val_mean_squared_error: nan - val_mean_absolute_error: nan
Epoch 2/100
153/153 [==============================] - 1s 4ms/step - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_mean_absolute_error: nan
Epoch 3/100
153/153 [==============================] - 1s 4ms/step - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_mean_absolute_error: nan
Epoch 4/100
153/153 [==============================] - 1s 4ms/step - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_mean_absolute_error: nan
Epoch 5/100
153/153 [==============================] - 1s 4ms/step - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - val_loss

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

153/153 [==============================] - 2s 8ms/step - loss: 7113983655936.0000 - mean_squared_error: 7113983655936.0000 - mean_absolute_error: 1699757.2500 - val_loss: 47239544176640.0000 - val_mean_squared_error: 47239544176640.0000 - val_mean_absolute_error: 6873085.0000
Epoch 2/100
153/153 [==============================] - 1s 7ms/step - loss: 4085098819354624.0000 - mean_squared_error: 4085098819354624.0000 - mean_absolute_error: 49374720.0000 - val_loss: 26043741085106176.0000 - val_mean_squared_error: 26043741085106176.0000 - val_mean_absolute_error: 161380752.0000
Epoch 3/100
153/153 [==============================] - 1s 6ms/step - loss: 1749817642585161728.0000 - mean_squared_error: 1749817642585161728.0000 - mean_absolute_error: 1026056192.0000 - val_loss: 11138129755184824320.0000 - val_mean_squared_error: 11138129755184824320.0000 - val_mean_absolute_error: 3337384960.0000
Epoch 4/100
153/153 [==============================] - 1s 6ms/step - loss: 748664469042724077568.000

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

153/153 [==============================] - 1s 7ms/step - loss: 39034206289920.0000 - mean_squared_error: 39034206289920.0000 - mean_absolute_error: 3085938.0000 - val_loss: 764995482681344.0000 - val_mean_squared_error: 764995482681344.0000 - val_mean_absolute_error: 27658548.0000
Epoch 2/100
153/153 [==============================] - 1s 4ms/step - loss: 2051519953181477961728.0000 - mean_squared_error: 2051519953181477961728.0000 - mean_absolute_error: 21504331776.0000 - val_loss: 42377503399269646729216.0000 - val_mean_squared_error: 42377503399269646729216.0000 - val_mean_absolute_error: 205857865728.0000
Epoch 3/100
153/153 [==============================] - 1s 4ms/step - loss: 113674690355453773983034376192.0000 - mean_squared_error: 113674690355453773983034376192.0000 - mean_absolute_error: 160074957848576.0000 - val_loss: 2348130620476170907477344256000.0000 - val_mean_squared_error: 2348130620476170907477344256000.0000 - val_mean_absolute_error: 1532361250439168.0000
Epoch 4/10

...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

153/153 [==============================] - 1s 5ms/step - loss: 182739173376.0000 - mean_squared_error: 182739173376.0000 - mean_absolute_error: 64800.2305 - val_loss: 1592653184.0000 - val_mean_squared_error: 1592653184.0000 - val_mean_absolute_error: 37896.1133
Epoch 2/100
153/153 [==============================] - 1s 4ms/step - loss: 181656961024.0000 - mean_squared_error: 181656961024.0000 - mean_absolute_error: 64777.8516 - val_loss: 322553440.0000 - val_mean_squared_error: 322553440.0000 - val_mean_absolute_error: 11758.5469
Epoch 3/100
153/153 [==============================] - 1s 4ms/step - loss: 181792456704.0000 - mean_squared_error: 181792456704.0000 - mean_absolute_error: 63926.2734 - val_loss: 1015012352.0000 - val_mean_squared_error: 1015012352.0000 - val_mean_absolute_error: 29680.1797
Epoch 4/100
153/153 [==============================] - 1s 4ms/step - loss: 180788592640.0000 - mean_squared_error: 180788592640.0000 - mean_absolute_error: 64477.5938 - val_loss: 225357824.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
6 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 767, in fit
    **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 918, in _fit
    X, y = self._initialize(X, y)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 855, in _initialize
  

229/229 [==============================] - 1s 4ms/step - loss: 94971387904.0000 - mean_squared_error: 94971387904.0000 - mean_absolute_error: 16806.4336 - val_loss: 220992416.0000 - val_mean_squared_error: 220992416.0000 - val_mean_absolute_error: 10777.2754
Epoch 2/50
229/229 [==============================] - 1s 4ms/step - loss: 94942593024.0000 - mean_squared_error: 94942593024.0000 - mean_absolute_error: 16663.1699 - val_loss: 221548112.0000 - val_mean_squared_error: 221548112.0000 - val_mean_absolute_error: 10883.2422
Epoch 3/50
229/229 [==============================] - 1s 4ms/step - loss: 94946164736.0000 - mean_squared_error: 94946164736.0000 - mean_absolute_error: 16629.2715 - val_loss: 221405024.0000 - val_mean_squared_error: 221405024.0000 - val_mean_absolute_error: 10859.0869
Epoch 4/50
229/229 [==============================] - 1s 4ms/step - loss: 94942429184.0000 - mean_squared_error: 94942429184.0000 - mean_absolute_error: 16519.4727 - val_loss: 221978544.0000 - val_mean

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(activation='sigmoid', epochs=100, hidden_layers_list=[256, 256, 256, 256], loss='mse', metrics=['mse', 'mae'], model=<function get_model_mlp_classifier at 0x7f611240f4d0>, n_layers=4),
                   param_distributions={'activation': ['relu', 'sigmoid',
                                                       'softmax'],
                                        'epochs': [50, 100],
                                        'hidden_layers_list': [[100, 100],
                                                               [200, 200],
                                                               [200, 200, 200],
                                                               [256, 256, 256],
                                                               [200, 200, 200,
                                                                200]],
                                        'n_layers': [2, 2, 3, 3, 4],
               

In [18]:
best_model_min_max_scaler = grid.best_estimator_

grid.best_params_

{'optimizer': 'sgd',
 'n_layers': 2,
 'hidden_layers_list': [100, 100],
 'epochs': 50,
 'activation': 'softmax'}

In [19]:
predictions_train_standard_scaler_keras_tuner = best_model_min_max_scaler.predict(X_train_one_hot_encoding_min_max)
predictions_val_standard_scaler_keras_tuner = best_model_min_max_scaler.predict(X_val_one_hot_encoding_min_max)

get_metrics(y_train_one_hot_encoding_min_max,predictions_train_standard_scaler_keras_tuner)
get_metrics(y_val_one_hot_encoding_min_max,predictions_val_standard_scaler_keras_tuner)

58/58 [==============================] - 0s 2ms/step
MAE:  14684.267325694826
RMSE:  308027.1983958738
r2 :  -0.00011419642130472418
MAE:  10676.442150705094
RMSE:  14854.44376670994
r2 :  -0.0004614569647716227


Con base a las métricas anteriores se observa evidencia de underfitting, ya que se obtuvo un desempeño menor en el conjunto de entrenamiento comparado con el de validación.

## Variación de hiperparámetros en dataset con columnas de Manufacturer y Model codificados con BinaryEncoder

In [5]:
X_train_binary_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/standard/train/X_train_binary_encoding_standard.csv')
y_train_binary_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/standard/train/Y_train_binary_encoding_standard.csv')

X_val_binary_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/standard/val/X_val_binary_encoding_standard.csv')
y_val_binary_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/standard/val/y_val_binary_encoding_standard.csv')

X_test_binary_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/standard/test/X_test_binary_encoding_standard.csv')
y_test_binary_encoding_standard = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/standard/test/Y_test_binary_encoding_standard.csv')


In [6]:
X_train_binary_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/min_max/train/X_train_binary_encoding_min_max.csv')
y_train_binary_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/min_max/train/y_train_binary_encoding_min_max.csv')

X_val_binary_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/min_max/val/X_val_binary_encoding_min_max.csv')
y_val_binary_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/min_max/val/y_val_binary_encoding_min_max.csv')

X_test_binary_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/min_max/test/X_test_binary_encoding_min_max.csv')
y_test_binary_encoding_min_max = pd.read_csv('https://raw.githubusercontent.com/MaldonadoVihlai/proyecto_CDA/main/Entrega_2/data/binary_encoding/min_max/test/y_test_binary_encoding_min_max.csv')


### Dataset con Standard Scaler

In [37]:
def get_model_mlp_classifier(meta, activation, n_layers, hidden_layers_list):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    dims = hidden_layers_list

    model = Sequential(name='Clasificador')

    #Capa de Entrada
    model.add(Input(shape=(X_shape_[1:]), name='Input_Layer'))


    #model.add(Dense(n_features_in_, input_shape=X_shape_[1:]))

    for i in range(n_layers):
        model.add(Dense(dims[i], activation=activation))
  
    model.add(Dense(1, activation='linear'))
    return model
#Búsqueda de hiperparámetros para el nuevo perceptrón multicapa    
final_params = {
    'hidden_layers_list':  [[100, 100], [200]*2, [200]*3, [256]*3, [200]*4],
    "epochs": [50],
    'optimizer': ["adam"],
    "activation": ["relu", "sigmoid", "softmax"],
    'n_layers':[2, 2, 3, 3, 4],
    #'batch_size':[10, 50, 100, 200]
}
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='/content/logs'),
]
clf = KerasRegressor(
    get_model_mlp_classifier,
    loss="mse",
    n_layers=4,
    metrics=["mse", "mae"],
    epochs=100,
    activation="sigmoid",
    hidden_layers_list = [256]*4
)

grid = RandomizedSearchCV(clf, final_params, cv=3, n_iter=10, random_state=33)
grid.fit(X_train_binary_encoding_standard,
        y_train_binary_encoding_standard,
        callbacks=my_callbacks,
         validation_data=(    
                X_val_binary_encoding_standard, 
                y_val_binary_encoding_standard
            )
)

Epoch 1/50
153/153 [==============================] - 1s 4ms/step - loss: 607402944.0000 - mean_squared_error: 607402944.0000 - mean_absolute_error: 18882.3594 - val_loss: 634798208.0000 - val_mean_squared_error: 634798208.0000 - val_mean_absolute_error: 19086.7988
Epoch 2/50
153/153 [==============================] - 1s 4ms/step - loss: 607389696.0000 - mean_squared_error: 607389696.0000 - mean_absolute_error: 18882.0098 - val_loss: 634784576.0000 - val_mean_squared_error: 634784576.0000 - val_mean_absolute_error: 19086.4395
Epoch 3/50
153/153 [==============================] - 1s 4ms/step - loss: 607375936.0000 - mean_squared_error: 607375936.0000 - mean_absolute_error: 18881.6504 - val_loss: 634770304.0000 - val_mean_squared_error: 634770304.0000 - val_mean_absolute_error: 19086.0664
Epoch 4/50
153/153 [==============================] - 1s 4ms/step - loss: 607361216.0000 - mean_squared_error: 607361216.0000 - mean_absolute_error: 18881.2598 - val_loss: 634755520.0000 - val_mean_squa

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
6 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 767, in fit
    **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 918, in _fit
    X, y = self._initialize(X, y)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 855, in _initialize
  

229/229 [==============================] - 1s 4ms/step - loss: 95351799808.0000 - mean_squared_error: 95351799808.0000 - mean_absolute_error: 22289.9629 - val_loss: 569370368.0000 - val_mean_squared_error: 569370368.0000 - val_mean_absolute_error: 17805.2734
Epoch 2/50
229/229 [==============================] - 1s 4ms/step - loss: 95165816832.0000 - mean_squared_error: 95165816832.0000 - mean_absolute_error: 18341.9492 - val_loss: 309517696.0000 - val_mean_squared_error: 309517696.0000 - val_mean_absolute_error: 11666.9736
Epoch 3/50
229/229 [==============================] - 1s 3ms/step - loss: 94979809280.0000 - mean_squared_error: 94979809280.0000 - mean_absolute_error: 14128.1729 - val_loss: 232172736.0000 - val_mean_squared_error: 232172736.0000 - val_mean_absolute_error: 9682.1201
Epoch 4/50
229/229 [==============================] - 1s 4ms/step - loss: 94929289216.0000 - mean_squared_error: 94929289216.0000 - mean_absolute_error: 12952.0518 - val_loss: 202021648.0000 - val_mean_

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(activation='sigmoid', epochs=100, hidden_layers_list=[256, 256, 256, 256], loss='mse', metrics=['mse', 'mae'], model=<function get_model_mlp_classifier at 0x7fa1d44da200>, n_layers=4),
                   param_distributions={'activation': ['relu', 'sigmoid',
                                                       'softmax'],
                                        'epochs': [50],
                                        'hidden_layers_list': [[100, 100],
                                                               [200, 200],
                                                               [200, 200, 200],
                                                               [256, 256, 256],
                                                               [200, 200, 200,
                                                                200]],
                                        'n_layers': [2, 2, 3, 3, 4],
                    

In [38]:
best_model_standar_scaler_binary = grid.best_estimator_

grid.best_params_

{'optimizer': 'adam',
 'n_layers': 2,
 'hidden_layers_list': [100, 100],
 'epochs': 50,
 'activation': 'relu'}

In [39]:
predictions_train_standard_scaler_keras_tuner = best_model_standar_scaler_binary.predict(X_train_binary_encoding_standard)
predictions_val_standard_scaler_keras_tuner = best_model_standar_scaler_binary.predict(X_val_binary_encoding_standard)

get_metrics(y_train_binary_encoding_standard,predictions_train_standard_scaler_keras_tuner)
get_metrics(y_val_binary_encoding_standard,predictions_val_standard_scaler_keras_tuner)

58/58 [==============================] - 0s 1ms/step
MAE:  11850.621862355094
RMSE:  307145.4137387866
r2 :  0.005474368979501487
MAE:  8537.36568733374
RMSE:  14288.03779797277
r2 :  0.24527278089819204


Con base a las métricas anteriores se observa evidencia de underfitting, ya que se obtuvo un desempeño menor en el conjunto de entrenamiento comparado con el de validación.

### Dataset con Min-Max Scaler

In [10]:
def get_model_mlp_classifier(meta, activation, n_layers, hidden_layers_list):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    dims = hidden_layers_list

    model = Sequential(name='Clasificador')

    #Capa de Entrada
    model.add(Input(shape=(X_shape_[1:]), name='Input_Layer'))


    #model.add(Dense(n_features_in_, input_shape=X_shape_[1:]))

    for i in range(n_layers):
        model.add(Dense(dims[i], activation=activation))
  
    model.add(Dense(1, activation='linear'))
    return model
#Búsqueda de hiperparámetros para el nuevo perceptrón multicapa    
final_params = {
    'hidden_layers_list':  [[200]*2, [200]*3, [256]*3, [200]*4],
    "epochs": [50],
    'optimizer': ["adam"],
    "activation": ["relu", "sigmoid", "softmax"],
    'n_layers':[2, 3, 3, 4],
    #'batch_size':[10, 50, 100, 200]
}
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='/content/logs'),
]
clf = KerasRegressor(
    get_model_mlp_classifier,
    loss="mse",
    n_layers=4,
    metrics=["mse", "mae"],
    epochs=100,
    activation="sigmoid",
    hidden_layers_list = [256]*4
)

grid = RandomizedSearchCV(clf, final_params, cv=3, n_iter=10, random_state=33)

grid.fit(X_train_binary_encoding_min_max,
        y_train_binary_encoding_min_max,
        callbacks=my_callbacks,
         validation_data=(    
                X_val_binary_encoding_min_max, 
                y_val_binary_encoding_min_max
            )
)

Epoch 1/50
153/153 [==============================] - 1s 5ms/step - loss: 142435663872.0000 - mean_squared_error: 142435663872.0000 - mean_absolute_error: 19235.6680 - val_loss: 293449216.0000 - val_mean_squared_error: 293449216.0000 - val_mean_absolute_error: 10722.2393
Epoch 2/50
153/153 [==============================] - 1s 4ms/step - loss: 142211284992.0000 - mean_squared_error: 142211284992.0000 - mean_absolute_error: 15941.2080 - val_loss: 275156128.0000 - val_mean_squared_error: 275156128.0000 - val_mean_absolute_error: 10355.8662
Epoch 3/50
153/153 [==============================] - 1s 4ms/step - loss: 142155399168.0000 - mean_squared_error: 142155399168.0000 - mean_absolute_error: 15648.7031 - val_loss: 266227888.0000 - val_mean_squared_error: 266227888.0000 - val_mean_absolute_error: 10192.4492
Epoch 4/50
153/153 [==============================] - 1s 4ms/step - loss: 142093156352.0000 - mean_squared_error: 142093156352.0000 - mean_absolute_error: 15160.0771 - val_loss: 281126

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
6 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 767, in fit
    **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 918, in _fit
    X, y = self._initialize(X, y)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 855, in _initialize
  

229/229 [==============================] - 1s 4ms/step - loss: 95133614080.0000 - mean_squared_error: 95133614080.0000 - mean_absolute_error: 19171.2422 - val_loss: 308639968.0000 - val_mean_squared_error: 308639968.0000 - val_mean_absolute_error: 10613.9102
Epoch 2/50
229/229 [==============================] - 1s 4ms/step - loss: 94877786112.0000 - mean_squared_error: 94877786112.0000 - mean_absolute_error: 14003.9219 - val_loss: 286144000.0000 - val_mean_squared_error: 286144000.0000 - val_mean_absolute_error: 10780.1807
Epoch 3/50
229/229 [==============================] - 1s 4ms/step - loss: 94856249344.0000 - mean_squared_error: 94856249344.0000 - mean_absolute_error: 13848.4883 - val_loss: 278821312.0000 - val_mean_squared_error: 278821312.0000 - val_mean_absolute_error: 11007.4316
Epoch 4/50
229/229 [==============================] - 1s 4ms/step - loss: 94837702656.0000 - mean_squared_error: 94837702656.0000 - mean_absolute_error: 13889.2627 - val_loss: 272013952.0000 - val_mean

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(activation='sigmoid', epochs=100, hidden_layers_list=[256, 256, 256, 256], loss='mse', metrics=['mse', 'mae'], model=<function get_model_mlp_classifier at 0x7fa1ea0fbef0>, n_layers=4),
                   param_distributions={'activation': ['relu', 'sigmoid',
                                                       'softmax'],
                                        'epochs': [50],
                                        'hidden_layers_list': [[200, 200],
                                                               [200, 200, 200],
                                                               [256, 256, 256],
                                                               [200, 200, 200,
                                                                200]],
                                        'n_layers': [2, 3, 3, 4],
                                        'optimizer': ['adam']},
                   random_state=33

In [ ]:
best_model_standar_scaler = grid.best_estimator_

grid.best_params_

In [12]:
predictions_train_min_max_scaler_keras_tuner = best_model_standar_scaler.predict(X_train_binary_encoding_min_max)
predictions_val_min_max_scaler_keras_tuner = best_model_standar_scaler.predict(X_val_binary_encoding_min_max)

get_metrics(y_train_binary_encoding_min_max,predictions_train_min_max_scaler_keras_tuner)
get_metrics(y_val_binary_encoding_min_max,predictions_val_min_max_scaler_keras_tuner)

58/58 [==============================] - 0s 1ms/step
MAE:  14295.886430148097
RMSE:  305122.8385653623
r2 :  0.018451882106095052
MAE:  10915.3093758089
RMSE:  18793.657798455646
r2 :  -0.16279054533776938


Con base a las métricas anteriores se observa evidencia de underfitting, ya que se obtuvo un desempeño menor en el conjunto de entrenamiento comparado con el de validación.

## Evaluación

Con base a los resultados observados en la variación de hiperparámetros, observando los resultados del RMSE en conjunto de validación encontramos que el mejor modelo es el que tiene las variables Manufacturer y Model, con codificación binaria, así como una transformación de Standard Scaler. Este modelo cuenta con:
* 'optimizer': 'adam',
* 'n_layers': 2,
* 'hidden_layers_list': [100, 100]
* 'epochs': 50,
* 'activation': 'relu' \

A continuación usaremos ese modelo para la predicción del conjunto de test con dicha transformación:

In [40]:
predictions_best_model = best_model_standar_scaler_binary.predict(X_test_binary_encoding_standard)
get_metrics(y_test_binary_encoding_standard, predictions_best_model)

72/72 [==============================] - 0s 1ms/step
MAE:  8604.738012526885
RMSE:  21672.64617800848
r2 :  0.21549254303945198


Con base en el resultado anterior se encuentra que probablemente el modelo tenga underfitting, ya que las métricas en entrenamiento fueron:
* MAE:  11850.621862355094
* RMSE:  307145.4137387866
* r2 :  0.005474368979501487 \
Y en validación teníamos: 
* MAE:  8537.36568733374
* RMSE:  14288.03779797277
* r2 :  0.24527278089819204

Como oportunidades de mejora del modelo, se considera que es posible añadir nuevas características sobre el conjunto de datos, dado que al añadir las variables de Manufacturer y Model, las métricas de error mejoraron considerablemente. Para el problema del underfitting se propone hacer una variación de hiperparámetros sobre un conjunto de valores más pequeño, cercano a los valores actuales, con el fin de sintonizar mejor el modelo y aumentar un poco su complejidad. Asimismo se propone hacer uso de k-fold validation para lograr tener un mejor entrenamiento. \
 Finalmente se propone probar nuevas arquitecturas de redes neuronales, así como otros tipos de redes neuronales como las redes neuronales convolucionales.